In [15]:
import pandas as pd
import numpy as np
import pickle
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
import keras_tuner

In [16]:
DATA_VERSION = "v2"

merged_data_path = f'../data/{DATA_VERSION}/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [17]:
data = data.sample(frac=0.01, random_state=42)

In [18]:
group_a_predictions = []
group_b_predictions = []

requests.delete("https://ium.rasztabiga.me/ab_test/results")

for index, row in data.iterrows():
    user_id = row ['user_id']
    genres = row['genres']
    favourite_genres = row['favourite_genres']
    skipped = row['skipped']

    group = user_id % 2

    prediction = requests.post(f"https://ium.rasztabiga.me/ab_test?user_id={user_id}", json={"genres": genres, "favourite_genres": favourite_genres}).json()

    if group == 0:
        group_a_predictions.append({"prediction": prediction, "skipped": skipped})
    else:
        group_b_predictions.append({"prediction": prediction, "skipped": skipped})

In [ ]:
print(group_a_predictions)
print(group_b_predictions)

group_a_accuracy = accuracy_score([x["skipped"] for x in group_a_predictions], [x["prediction"] for x in group_a_predictions])
group_b_accuracy = accuracy_score([x["skipped"] for x in group_b_predictions], [x["prediction"] for x in group_b_predictions])]

print(group_a_accuracy)
print(group_b_accuracy)